In [1]:
# import sys
# sys.path.insert(0, '..')
from semantic_mpc_interface import (
    BuildingMetadataLoader,
    HPFlexSurvey,
    convert_units,
    SHACLHandler
)
from buildingmotif.namespaces import BRICK, RDF
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


# Testing the Model Builder

ModelBuilder has been deprecated (and will be replaced in the future)

# Testing the Survey Workflow

In [2]:
# Please disregad excessive logs for now 

# Creating survey, allow overwrite if there is something already there
s = HPFlexSurvey('test_site','test_build','.', overwrite=True)

# Generating a simple building structure that prefills csv files. 
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])
# Now check test_site/test_build to look at survey files


DEBUG:buildingmotif.database.graph_connection:Creating tables for graph storage


DEBUG:buildingmotif.database.table_connection:Creating shape collection in model: 'urn:hpflex/test_site#'
DEBUG:buildingmotif.database.table_connection:Creating model: 'urn:hpflex/test_site#', with graph: '6195e130-6e0c-462f-b4de-065a1a711bb1'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '6195e130-6e0c-462f-b4de-065a1a711bb1' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating shape collection in library: 'brick-templates'
DEBUG:buildingmotif.database.table_connection:Creating database library: 'brick-templates'
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hvac-zone'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '99b12d33-2b74-4b90-b105-f4e28f19479a' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'space'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '0bdf26f3-b1b1-4b80-b600-ec20615627b6' in database with: 2 triples
DEBUG:bu

removing dependency:  space
removing dependency:  window
removing dependency:  hp-rtu
removing dependency:  tstat
values: dict_values([])
values names: {}
values: dict_values(['area-name-value'])
values names: {'area-name': '<name>-area-name'}
values: dict_values(['heating_COP-name-value', 'cooling_COP-name-value', 'cooling_capacity-name-value', 'heating_capacity-name-value'])
values names: {'cooling_COP-name': '<name>-cooling_COP-name', 'heating_COP-name': '<name>-heating_COP-name', 'cooling_capacity-name': '<name>-cooling_capacity-name', 'heating_capacity-name': '<name>-heating_capacity-name'}
values: dict_values(['resolution-name-value', 'stage_count-name-value', 'active-name-value', 'setpoint_deadband-name-value', 'tolerance-name-value'])
values names: {'tolerance-name': '<name>-tolerance-name', 'resolution-name': '<name>-resolution-name', 'active-name': '<name>-active-name', 'stage_count-name': '<name>-stage_count-name', 'setpoint_deadband-name': '<name>-setpoint_deadband-name'}
v

In [3]:
# Will just fill the columns programmatically for testing, csv should be filled out otherwise
import sys
sys.path.insert(0,'../examples')
from example_prefill_usage import prefill_csv_survey

In [4]:
prefill_csv_survey('test_site/test_build')

Loading config from: test_site/test_build/config.json
Found CSV files: ['zone', 'site', 'window', 'hvac', 'point_list', 'tstat', 'space']
Filled 4 empty cells across 4 columns in site.csv
Filled 21 empty cells across 3 columns in window.csv
Filled 12 empty cells across 4 columns in hvac.csv
No empty cells found in point_list.csv
Filled 15 empty cells across 5 columns in tstat.csv
Filled 8 empty cells across 2 columns in space.csv


In [5]:
# Reading csv
s.read_csv()

Expanded CSV shape: (6, 5)
New columns added: []
Expanded CSV shape: (4, 4)
New columns added: ['area-name']
Expanded CSV shape: (3, 9)
New columns added: ['cooling_COP-name', 'heating_COP-name', 'cooling_capacity-name', 'heating_capacity-name']
Expanded CSV shape: (3, 14)
New columns added: ['tolerance-name', 'resolution-name', 'active-name', 'stage_count-name', 'setpoint_deadband-name']
Expanded CSV shape: (7, 8)
New columns added: ['area-name', 'azimuth-name', 'tilt-name']
Expanded CSV shape: (1, 9)
New columns added: ['latitude-name', 'longitude-name', 'noaastation-name', 'timezone-name']


# Testing SHACL Generation and Validation

In [6]:
og = clone.clone_graph(s.graph)

In [7]:
# Create handler
handler = SHACLHandler()

# Generate shapes
handler.generate_shapes()

# Save shapes
handler.save_shapes('shapes.ttl')

# Validate a model
conforms, results_graph, results_text = handler.validate_model(s.graph)

s.graph.serialize('test-brick-model-reasoned.ttl', format = 'ttl')
if not conforms:
    print("Validation failed:")
    print(results_text)

In [8]:
# lots of new inferred information
(s.graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/test_site#hvac_1> ns1:air-connects-to <urn:hpflex/test_site#zone_1> .

<urn:hpflex/test_site#hvac_2> ns1:air-connects-to <urn:hpflex/test_site#zone_2> .

<urn:hpflex/test_site#hvac_3> ns1:air-connects-to <urn:hpflex/test_site#zone_3> .

<urn:hpflex/test_site#tstat_zone_1> ns1:has-location <urn:hpflex/test_site#zone_1> ;
    ns1:has-point <urn:hpflex/test_site#tstat_zone_1-active-name>,
        <urn:hpflex/test_site#tstat_zone_1-setpoint_deadband-name>,
        <urn:hpflex/test_site#tstat_zone_1-tolerance-name> .

<urn:hpflex/test_site#tstat_zone_2> ns1:has-location <urn:hpflex/test_site#zone_2> ;
    ns1:has-point <urn:hpflex/test_site#tstat_zone_2-active-name>,
        <urn:hpflex/test_site#tstat_zone_2-setpoint_deadband-name>,
        <urn:hpflex/test_site#tstat_zone_2-tolerance-name> .

<urn:hpflex/test_site#tstat_zone_3> ns1:has-location <urn:hpflex/test_site#zone_3> ;
    ns1:has-point <urn:hpflex/test_site#tstat_zone_3-active-name>

# Testing get Metadata

In [9]:
site_metadata = BuildingMetadataLoader("test_site/test_build/test_build.ttl", ontology = 'brick')
site_info = site_metadata.get_site_info()
thermostat_data = site_metadata.get_thermostat_data()

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
thermostat_data

{'heat_availability': [False, False],
 'cool_availability': [True, True],
 'heat_tolerance': [-2.0, -2.0],
 'cool_tolerance': [2.0, 2.0],
 'setpoint_deadband': [1, 1],
 'active': [True, True],
 'control_group': ['DEPRECATED', 'DEPRECATED'],
 'control_type_list': ['stage', 'stage'],
 'floor_area_list': [50.0, 50.0],
 'window_area_list': [10.5, 10.5],
 'azimuth_list': [180, 180],
 'tilt_list': [30, 30],
 'zone_ids': ['zone1', 'zone2'],
 'hvacs': ['hvac1', 'hvac2'],
 'setpoint_type': ['double', 'double'],
 'fuel_heat_list': ['gas', 'gas'],
 'fuel_cool_list': ['electricity', 'electricity'],
 'cooling_capacity': [5.0, 5.0],
 'heating_capacity': [4.0, 4.0],
 'cooling_cop': [3.5, 3.5],
 'heating_cop': [3.0, 3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [1, 1],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F'),
  rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F')]}

In [ ]:
site_metadata.get_thermostat_data(for_zone='zone1')

{'heat_availability': [False],
 'cool_availability': [True],
 'heat_tolerance': [-2.0],
 'cool_tolerance': [2.0],
 'setpoint_deadband': [1],
 'active': [True],
 'control_group': ['DEPRECATED'],
 'control_type_list': ['stage'],
 'floor_area_list': [50.0],
 'window_area_list': [10.5],
 'azimuth_list': [180],
 'tilt_list': [30],
 'zone_ids': ['zone1'],
 'hvacs': ['hvac1'],
 'setpoint_type': ['double'],
 'fuel_heat_list': ['gas'],
 'fuel_cool_list': ['electricity'],
 'cooling_capacity': [5.0],
 'heating_capacity': [4.0],
 'cooling_cop': [3.5],
 'heating_cop': [3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [1],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F')]}

In [ ]:
site_metadata.get_complete_output()

{'tz': 'urn:hpflex/test-site#test-site.timezone',
 'latitude': 'urn:hpflex/test-site#test-site.latitude',
 'longitude': 'urn:hpflex/test-site#test-site.longitude',
 'NOAAstation': 'urn:hpflex/test-site#test-site.noaastation',
 'project_id': rdflib.term.URIRef('urn:hpflex/test-site#test-site'),
 'site_id': rdflib.term.URIRef('urn:hpflex/test-site#test-site'),
 'heat_availability': [False, False],
 'cool_availability': [True, True],
 'heat_tolerance': [-2.0, -2.0],
 'cool_tolerance': [2.0, 2.0],
 'setpoint_deadband': [1, 1],
 'active': [True, True],
 'control_group': ['DEPRECATED', 'DEPRECATED'],
 'control_type_list': ['stage', 'stage'],
 'floor_area_list': [50.0, 50.0],
 'window_area_list': [10.5, 10.5],
 'azimuth_list': [180, 180],
 'tilt_list': [30, 30],
 'zone_ids': ['zone1', 'zone2'],
 'hvacs': ['hvac1', 'hvac2'],
 'setpoint_type': ['double', 'double'],
 'fuel_heat_list': ['gas', 'gas'],
 'fuel_cool_list': ['electricity', 'electricity'],
 'cooling_capacity': [5.0, 5.0],
 'heating_ca

# Testing Unit Conversion

In [ ]:
convert_units(10, 'FT', 'M')

3.048

In [ ]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

In [ ]:
convert_units(0, 'DEG_C', 'K')

273.15

In [ ]:
site_metadata.convert_model_to_si()
site_metadata.get_thermostat_data(for_zone='zone1')

changing value of  urn:hpflex/test-site#window1_area from http://qudt.org/vocab/unit/FT2 to http://qudt.org/vocab/unit/M2
changing value of  urn:hpflex/test-site#window2_area from http://qudt.org/vocab/unit/FT2 to http://qudt.org/vocab/unit/M2


{'heat_availability': [False],
 'cool_availability': [True],
 'heat_tolerance': [-2.0],
 'cool_tolerance': [2.0],
 'setpoint_deadband': [1],
 'active': [True],
 'control_group': ['DEPRECATED'],
 'control_type_list': ['stage'],
 'floor_area_list': [50.0],
 'window_area_list': [0.9754819200000001],
 'azimuth_list': [180],
 'tilt_list': [30],
 'zone_ids': ['zone1'],
 'hvacs': ['hvac1'],
 'setpoint_type': ['double'],
 'fuel_heat_list': ['gas'],
 'fuel_cool_list': ['electricity'],
 'cooling_capacity': [5.0],
 'heating_capacity': [4.0],
 'cooling_cop': [3.5],
 'heating_cop': [3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [1],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F')]}

In [ ]:
"""
The temperature unit still shows up as DEG_F, because it is at the units for the zone temperature, 
which does not have a value to convert in the model, it's values are in the DB.
it is a reminder to also convert that to C also. We can use the unit_conversion script for any conversions in the db.
"""

"\nThe temperature unit still shows up as DEG_F, because it is at the units for the zone temperature, \nwhich does not have a value to convert in the model, it's values are in the DB.\nit is a reminder to also convert that to C also. We can use the unit_conversion script for any conversions in the db.\n"

# Testing Grafana Dashboarding 

In [ ]:
# import yaml 
# with open('../development_files/creds.yml') as f:
#     config = yaml.safe_load(f)

# bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

In [ ]:
# bg.create_dashboard('AQL from brick')

Dashboard(title='AQL from brick', annotations=Annotations(list=[]), description='Zone-based dashboard generated from Brick model', editable=True, gnetId=None, graphTooltip=0, hideControls=False, id=None, inputs=[], links=[], panels=[], refresh='10s', rows=[], schemaVersion=12, sharedCrosshair=False, style='dark', tags=['brick'], templating=Templating(list=[]), time=Time(start='now-1h', end='now'), timePicker=TimePicker(refreshIntervals=['5s', '10s', '30s', '1m', '5m', '15m', '30m', '1h', '2h', '1d'], timeOptions=['5m', '15m', '1h', '6h', '12h', '24h', '2d', '7d', '30d'], nowDelay=None, hidden=False), timezone='browser', version=0, uid=None)

In [ ]:
# Grafana not set up, upload won't work
# bg.upload_dashboard(message = 'testing upload')